## 1. Load Datasets

In [1]:
## Utils
import re
import numpy as np
import pandas as pd
import itertools

## Classical Learner
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_validate

In [2]:
from sklearn.base import clone


class OrdinalClassifier():
    
    def __init__(self, clf):
        self.clf = clf
        self.clfs = {}
    
    def fit(self, X, y):
        self.unique_class = np.sort(np.unique(y))
        if self.unique_class.shape[0] > 2:
            for i in range(self.unique_class.shape[0]-1):
                # for each k - 1 ordinal value we fit a binary classification problem
                binary_y = (y > self.unique_class[i]).astype(np.uint8)
                clf = clone(self.clf)
                clf.fit(X, binary_y)
                self.clfs[i] = clf
    
    def predict_proba(self, X):
        clfs_predict = {k:self.clfs[k].predict_proba(X) for k in self.clfs}
        predicted = []
        for i,y in enumerate(self.unique_class):
            if i == 0:
                # V1 = 1 - Pr(y > V1)
                predicted.append(1 - clfs_predict[y][:,1])
            elif y in clfs_predict:
                # Vi = Pr(y > Vi-1) - Pr(y > Vi)
                 predicted.append(clfs_predict[y-1][:,1] - clfs_predict[y][:,1])
            else:
                # Vk = Pr(y > Vk-1)
                predicted.append(clfs_predict[y-1][:,1])
        return np.vstack(predicted).T
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

In [3]:
"""Dataset Parameter Setting"""
"""Load Dataset"""
r6g_dset = pd.read_csv("C:/Users/sypark/Desktop/Projects/w_MinSeok/1SERSNet/2data/_preprocessed/sersnet_devset.csv")
r6g_tset1 = pd.read_csv("C:/Users/sypark/Desktop/Projects/w_MinSeok/1SERSNet/2data/_preprocessed/sersnet_batch1_devset.csv")
r6g_tset2 = pd.read_csv("C:/Users/sypark/Desktop/Projects/w_MinSeok/1SERSNet/2data/_preprocessed/sersnet_devse_batch3_1.csv")
r6g_tset3 = pd.read_csv("C:/Users/sypark/Desktop/Projects/w_MinSeok/1SERSNet/2data/_preprocessed/sersnet_devse_batch3_2.csv")
r6g_tset4 = pd.read_csv("C:/Users/sypark/Desktop/Projects/w_MinSeok/1SERSNet/2data/_preprocessed/sersnet_devse_batch3_3.csv")

"""Set Output Path"""
fileout = "C:/Users/sypark/Desktop/Projects/w_MinSeok/1SERSNet/3results/R6G_Classification/raw_data/baseline_r6g_ordinal_model_output_comb.csv"

## Class Definition
## 2: Conc. >= 10 mM  
## 1: 10 uM <= Conc. < 10 mM
## 0: Conc. < 10 uM

In [4]:
r6g_dset = r6g_dset.iloc[:,1:]
r6g_tset1 = r6g_tset1.iloc[:,2:]

In [5]:
r6g_all_dset = pd.concat([r6g_dset, r6g_tset1, r6g_tset2, r6g_tset3, r6g_tset4], axis=0).reset_index(drop=True)

In [6]:
X_dset = r6g_all_dset.iloc[:, 1:].to_numpy(dtype='float32')
y_dset = r6g_all_dset.iloc[:,0].to_numpy(dtype='int64') 

## 2. Train Test Split

In [7]:
## 4.4 train and test split for Classical Learner
X_train, X_test, y_train, y_test = train_test_split(X_dset, y_dset, test_size = 0.2, 
                                                    random_state=123)

## 3. Classical Learner

### 3.1 Naive Bayes

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import BernoulliNB

In [9]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(BernoulliNB())
scaler.fit(X_train)
X_train_sds = scaler.transform(X_train)
X_test_sds = scaler.transform(X_test)
clf.fit(X_train_sds, y_train)

In [10]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[491   4   0]
 [201 320   0]
 [ 82   5   8]]
Metrics
ACC: 0.737
BACC: 0.563
F1_micro: 0.737
F1_macro: 0.561
AUROC_OVR: 0.739
AUROC_OVO: 0.709
Precisio_micro: 0.737
Precisio_macro: 0.869
Recall_micro: 0.737
Recall_macro: 0.563


In [11]:
NB_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [12]:
outF = open(fileout, "w")
outF.write("Naive_Bayes, ")
outF.write("ACC, BACC, F1_micro, F1_macro, AUROC_OVR, AUROC_OVO, Precision_micro, Precision_macro, Recall_micro, Recall_macro\n")
outF.write('DevSet, ')
outF.write(', '.join(map(str, NB_devset_res)))
outF.write('\n')
outF.close()

### 3.2 Decision Tree

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

In [14]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(DecisionTreeClassifier())
scaler.fit(X_train)
X_train_sds = scaler.transform(X_train)
X_test_sds = scaler.transform(X_test)
clf.fit(X_train_sds, y_train)

In [15]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[495   0   0]
 [  0 520   1]
 [  0   3  92]]
Metrics
ACC: 0.996
BACC: 0.989
F1_micro: 0.996
F1_macro: 0.992
AUROC_OVR: 0.993
AUROC_OVO: 0.992
Precisio_micro: 0.996
Precisio_macro: 0.995
Recall_micro: 0.996
Recall_macro: 0.989


In [16]:
DT_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [17]:
outF = open(fileout, "a")
outF.write("Decision_Tree, ")
outF.write("ACC, BACC, F1_micro, F1_macro, AUROC_OVR, AUROC_OVO, Precision_micro, Precision_macro, Recall_micro, Recall_macro\n")
outF.write('DevSet, ')
outF.write(', '.join(map(str, DT_devset_res)))
outF.write('\n')
outF.close()

### 3.3 Logistic Regression Classifier

In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [19]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(LogisticRegression())
scaler.fit(X_train)
X_train_sds = scaler.transform(X_train)
X_test_sds = scaler.transform(X_test)
clf.fit(X_train_sds, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

In [20]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[495   0   0]
 [  0 516   5]
 [  0   7  88]]
Metrics
ACC: 0.989
BACC: 0.972
F1_micro: 0.989
F1_macro: 0.975
AUROC_OVR: 0.997
AUROC_OVO: 0.996
Precisio_micro: 0.989
Precisio_macro: 0.978
Recall_micro: 0.989
Recall_macro: 0.972


In [21]:
LR_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [22]:
outF = open(fileout, "a")
outF.write("Logistic_Regression, ")
outF.write("ACC, BACC, F1_micro, F1_macro, AUROC_OVR, AUROC_OVO, Precision_micro, Precision_macro, Recall_micro, Recall_macro\n")
outF.write('DevSet, ')
outF.write(', '.join(map(str, LR_devset_res)))
outF.write('\n')
outF.close()

### 3.3 Logistic Regresssion CV

In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV

In [24]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(LogisticRegressionCV())
scaler.fit(X_train)
X_train_sds = scaler.transform(X_train)
X_test_sds = scaler.transform(X_test)
clf.fit(X_train_sds, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

In [25]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[495   0   0]
 [  0 517   4]
 [  0   3  92]]
Metrics
ACC: 0.994
BACC: 0.987
F1_micro: 0.994
F1_macro: 0.986
AUROC_OVR: 0.998
AUROC_OVO: 0.998
Precisio_micro: 0.994
Precisio_macro: 0.984
Recall_micro: 0.994
Recall_macro: 0.987


In [26]:
LR_CV_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [27]:
outF = open(fileout, "a")
outF.write("Logistic_Regression_CV, ")
outF.write("ACC, BACC, F1_micro, F1_macro, AUROC_OVR, AUROC_OVO, Precision_micro, Precision_macro, Recall_micro, Recall_macro\n")
outF.write('DevSet, ')
outF.write(', '.join(map(str, LR_CV_devset_res)))
outF.write('\n')
outF.close()

### 3.4 MLP

In [28]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

In [29]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(MLPClassifier())
scaler.fit(X_train)
X_train_sds = scaler.transform(X_train)
X_test_sds = scaler.transform(X_test)
clf.fit(X_train_sds, y_train)

In [30]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[495   0   0]
 [  0 519   2]
 [  0   1  94]]
Metrics
ACC: 0.997
BACC: 0.995
F1_micro: 0.997
F1_macro: 0.994
AUROC_OVR: 0.996
AUROC_OVO: 0.996
Precisio_micro: 0.997
Precisio_macro: 0.992
Recall_micro: 0.997
Recall_macro: 0.995


In [31]:
MLP_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [32]:
outF = open(fileout, "a")
outF.write("MLP, ")
outF.write("ACC, BACC, F1_micro, F1_macro, AUROC_OVR, AUROC_OVO, Precision_micro, Precision_macro, Recall_micro, Recall_macro\n")
outF.write('DevSet, ')
outF.write(', '.join(map(str, MLP_devset_res)))
outF.write('\n')
outF.close()

### 3.5 Random Forest

In [33]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [34]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(RandomForestClassifier())
scaler.fit(X_train)
X_train_sds = scaler.transform(X_train)
X_test_sds = scaler.transform(X_test)
clf.fit(X_train_sds, y_train)

In [35]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[495   0   0]
 [  0 521   0]
 [  0   0  95]]
Metrics
ACC: 1.0
BACC: 1.0
F1_micro: 1.0
F1_macro: 1.0
AUROC_OVR: 1.0
AUROC_OVO: 1.0
Precisio_micro: 1.0
Precisio_macro: 1.0
Recall_micro: 1.0
Recall_macro: 1.0


In [36]:
RF_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [37]:
outF = open(fileout, "a")
outF.write("Random_Forest, ")
outF.write("ACC, BACC, F1_micro, F1_macro, AUROC_OVR, AUROC_OVO, Precision_micro, Precision_macro, Recall_micro, Recall_macro\n")
outF.write('DevSet, ')
outF.write(', '.join(map(str, RF_devset_res)))
outF.write('\n')
outF.close()

### 3.6 Linear SVM

In [38]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [39]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(SVC(kernel = 'linear', probability=True))
scaler.fit(X_train)
X_train_sds = scaler.transform(X_train)
X_test_sds = scaler.transform(X_test)
clf.fit(X_train_sds, y_train)

In [40]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[495   0   0]
 [  0 518   3]
 [  0   2  93]]
Metrics
ACC: 0.995
BACC: 0.991
F1_micro: 0.995
F1_macro: 0.99
AUROC_OVR: 0.999
AUROC_OVO: 0.998
Precisio_micro: 0.995
Precisio_macro: 0.988
Recall_micro: 0.995
Recall_macro: 0.991


In [41]:
LinSVM_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [42]:
outF = open(fileout, "a")
outF.write("Linear_SVM, ")
outF.write("ACC, BACC, F1_micro, F1_macro, AUROC_OVR, AUROC_OVO, Precision_micro, Precision_macro, Recall_micro, Recall_macro\n")
outF.write('DevSet, ')
outF.write(', '.join(map(str, LinSVM_devset_res)))
outF.write('\n')
outF.close()

### 3.7 RBF SVM (Nonlinear SVM)

In [43]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [44]:
## Learn Classifier
scaler = StandardScaler()
clf = OrdinalClassifier(SVC(kernel = 'rbf', probability=True))
scaler.fit(X_train)
X_train_sds = scaler.transform(X_train)
X_test_sds = scaler.transform(X_test)
clf.fit(X_train_sds, y_train)

In [45]:
### Test within batch
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
yp_test = clf.predict(X_test_sds)
ys_test = clf.predict_proba(X_test_sds)

print('\033[1m' + 'Confusion Matrix' + '\033[0m')
print(confusion_matrix(y_test, yp_test))

print('\033[1m' + 'Metrics' + '\033[0m')
print('ACC: {}'.format(round(accuracy_score(y_test, yp_test), 3)))
print('BACC: {}'.format(round(balanced_accuracy_score(y_test, yp_test), 3)))
print('F1_micro: {}'.format(round(f1_score(y_test, yp_test, average='micro'),3)))
print('F1_macro: {}'.format(round(f1_score(y_test, yp_test, average='macro'),3)))
print('AUROC_OVR: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3)))
print('AUROC_OVO: {}'.format(round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3)))
print('Precisio_micro: {}'.format(round(precision_score(y_test, yp_test, average='micro'),3)))
print('Precisio_macro: {}'.format(round(precision_score(y_test, yp_test, average='macro'),3)))
print('Recall_micro: {}'.format(round(recall_score(y_test, yp_test, average='micro'),3)))
print('Recall_macro: {}'.format(round(recall_score(y_test, yp_test, average='macro'),3)))

Confusion Matrix
[[495   0   0]
 [  0 520   1]
 [  1   0  94]]
Metrics
ACC: 0.998
BACC: 0.996
F1_micro: 0.998
F1_macro: 0.996
AUROC_OVR: 0.996
AUROC_OVO: 0.996
Precisio_micro: 0.998
Precisio_macro: 0.996
Recall_micro: 0.998
Recall_macro: 0.996


In [46]:
RBFSVM_devset_res = [round(accuracy_score(y_test, yp_test), 3), round(balanced_accuracy_score(y_test, yp_test), 3),
                 round(f1_score(y_test, yp_test, average='micro'),3), round(f1_score(y_test, yp_test, average='macro'),3),
                 round(roc_auc_score(y_test, ys_test, multi_class='ovr'),3), round(roc_auc_score(y_test, ys_test, multi_class='ovo'),3),
                 round(precision_score(y_test, yp_test, average='micro'),3), round(precision_score(y_test, yp_test, average='macro'),3),
                 round(recall_score(y_test, yp_test, average='micro'),3), round(recall_score(y_test, yp_test, average='macro'),3)]

In [47]:
outF = open(fileout, "a")
outF.write("RBF_SVM, ")
outF.write("ACC, BACC, F1_micro, F1_macro, AUROC_OVR, AUROC_OVO, Precision_micro, Precision_macro, Recall_micro, Recall_macro\n")
outF.write('DevSet, ')
outF.write(', '.join(map(str, RBFSVM_devset_res)))
outF.write('\n')
outF.close()